In [2]:
import SimpleITK as sitk
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
from IPython.display import clear_output
import pprint
import os, sys
import pandas_profiling
import sys
import json
from scipy import ndimage
import nibabel as nib 
from pathlib import Path
sys.path.insert(0, str(Path(r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\src\registration_metric.pyy').resolve().parent.parent))
from src.eda_nifti import *
from src.registration_metric import *

In [3]:
def recursive_glob_files(top_folder_path, file_format):
    gathered_file_pathes = []

    for (root, directories, files) in os.walk(top_folder_path):
        for file in files:
            if file_format in file:
                detect_file_path = os.path.join(root, file)
                gathered_file_pathes.append(detect_file_path)

    return gathered_file_pathes

In [4]:
def display_images(fixed_image_z, moving_image_z, fixed_npa, moving_npa):
    # Create a figure with two subplots and the specified size.
    plt.subplots(1,2,figsize=(10,8))
   
    # Draw the fixed image in the first subplot.
    plt.subplot(1,2,1)
    plt.imshow(fixed_npa[fixed_image_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('fixed image')
    plt.axis('off')
   
    # Draw the moving image in the second subplot.
    plt.subplot(1,2,2)
    plt.imshow(moving_npa[moving_image_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('moving image')
    plt.axis('off')
   
    plt.show()

In [5]:
def display_images_with_alpha(image_z, alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[:,:,image_z] + alpha*moving[:,:,image_z] 
    plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()

# 0. Data Definition and Tree

- task1: ISCHEMIC STROKE LESION (multi-modal) adc, dwi, flair
- task2: ATLAS 2.0 (uni-modal) t1

## task1: In the folder at the bottom, there are 3 nifti images each and a json file containing meta information.

In [116]:
raw_data_folder_path = r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task1\rawdata'

In [117]:
raw_nifti_data_pathes = recursive_glob_files(raw_data_folder_path, '.nii.gz')
raw_nifti_meda_json_data_pathes = recursive_glob_files(raw_data_folder_path, '.json')

In [118]:
print(f'task1: raw_data total_count: {len(raw_nifti_data_pathes)}')
print(f'task1: raw_nifti_meda_json_data total_count: {len(raw_nifti_meda_json_data_pathes)}')

task1: raw_data total_count: 750
task1: raw_nifti_meda_json_data total_count: 432


In [119]:
total_json = dict()

for i, json_file_path in enumerate(raw_nifti_meda_json_data_pathes):
    with open(json_file_path) as json_file:
        json_data = json.load(json_file)
   
    total_json[json_file_path] = json_data


In [129]:
df_total_json = pd.DataFrame(total_json).T

In [130]:
df_total_json.reset_index(inplace=True)

In [131]:
df_total_json['subject'] = [ x.split('\\')[-3] for x in df_total_json['index']]

In [132]:
df_total_json['modality'] = [ x.split('\\')[-1].split('_')[-1].rstrip('.json') for x in df_total_json['index']]

In [133]:
df_total_json.drop(['index'], axis=1, inplace=True)

In [134]:
df_total_json = df_total_json[['dataset', 'subject', 'modality', 'ImageType', 'Modality', 'Manufacturer',
       'ManufacturerModelName', 'PatientSex', 'PatientAge', 'PatientWeight',
       'BodyPartExamined', 'ScanningSequence', 'SequenceVariant',
       'ScanOptions', 'MRAcquisitionType', 'SliceThickness', 'RepetitionTime',
       'EchoTime', 'NumberOfAverages', 'ImagingFrequency', 'ImagedNucleus',
       'EchoNumbers', 'MagneticFieldStrength', 'SpacingBetweenSlices',
       'NumberOfPhaseEncodingSteps', 'EchoTrainLength', 'PercentSampling',
       'PercentPhaseFieldOfView', 'PixelBandwidth', 'ReconstructionDiameter',
       'ReceiveCoilName', 'AcquisitionMatrix', 'FlipAngle', 'PatientPosition',
       'AcquisitionDuration', 'DiffusionBValue',
       'DiffusionGradientOrientation', 'ImagePositionPatient',
       'ImageOrientationPatient', 'Rows', 'Columns', 'InversionTime',
       ]]

In [136]:
df_total_json.to_csv("task1_meta_data.csv", encoding='ansi', index=False)

In [163]:
pr_of_task1=df_total_json.profile_report() # 프로파일링 결과 리포트를 pr에 저장
pr_of_task1.to_file('./pr_report_of_task1.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## task2: meta data excel profiling

In [155]:
!pip install openpyxl

In [158]:
df_metadata_of_task2 = pd.read_excel(r"C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task2\20220425_ATLAS_2.0_MetaData.xlsx")

In [162]:
pr_of_task2=df_metadata_of_task2.profile_report() # 프로파일링 결과 리포트를 pr에 저장
pr_of_task2.to_file('./pr_report_of_task2.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# 1. Make Meta dataframe using nifti file header

- task1: ISCHEMIC STROKE LESION (multi-modal == adc, dwi, flair)
- task2: ATLAS 2.0 (uni-modal == t1)

In [ ]:
task1_nifti_files_list = nifti_anal.recursive_find_all_files(r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task1', '.nii.gz')

nifti_anal.save_summary_table(task1_nifti_files_list, 'task1.csv', 'task1')

In [ ]:
task1_nifti_files_list = nifti_anal.recursive_find_all_files(r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task2', '.nii.gz')

nifti_anal.save_summary_table(task1_nifti_files_list, 'task2.csv', 'task2')

# 2. Co-registered data?

### 2.1 정성적 평가
- 결과: adc, dwi는 spacing도 동일하고 정성적으로 Co-registered data로 판단할 수 있음
- flair의 경우 다음과 같은 오류가 발생함

The "image2" input may need casting to the "64-bit float" pixel type.

In [43]:
fixed_image = sitk.ReadImage(r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task1\rawdata\sub-strokecase0001\ses-0001\sub-strokecase0001_ses-0001_adc.nii.gz')
moving_image = sitk.ReadImage(r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task1\rawdata\sub-strokecase0001\ses-0001\sub-strokecase0001_ses-0001_dwi.nii.gz')

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2]-1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image), moving=fixed(moving_image))

interactive(children=(IntSlider(value=36, description='image_z', max=72), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

In [44]:
fixed_image = sitk.ReadImage(r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task1\rawdata\sub-strokecase0001\ses-0001\sub-strokecase0001_ses-0001_adc.nii.gz')
moving_image = sitk.ReadImage(r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task1\rawdata\sub-strokecase0001\ses-0001\sub-strokecase0001_ses-0001_flair.nii.gz')

interact(display_images_with_alpha, image_z=(0,fixed_image.GetSize()[2]-1), alpha=(0.0,1.0,0.05), fixed = fixed(fixed_image), moving=fixed(moving_image))

interactive(children=(IntSlider(value=36, description='image_z', max=72), FloatSlider(value=0.5, description='…

<function __main__.display_images_with_alpha(image_z, alpha, fixed, moving)>

### 2.2 정량적 평가


In [21]:
fixed_image_path = r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task1\rawdata\sub-strokecase0001\ses-0001\sub-strokecase0001_ses-0001_adc.nii.gz'
moving_image_path = r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task1\rawdata\sub-strokecase0001\ses-0001\sub-strokecase0001_ses-0001_dwi.nii.gz'

In [22]:
registerd_metric = RegistrationMetric()

In [23]:
registerd_metric.calculate_registration_metric(fixed_image_path, moving_image_path)

{'mutual_info': -0.467,
 'ANTS_neighborhood_corr': -0.9103,
 'corr': -0.3931,
 'joint_histogram_mutualInfo': -1.2447,
 'mean_square': 8813.6888}

# 3. calculate missing volume size of task traing dataset (r29) atlas 2.0

In [165]:
r29_mask = r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task2\Training\R029\sub-r029s009\ses-1\anat\sub-r029s009_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz'

In [171]:
task2_train_data_path = r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task2\Training'

task2_train_data_path_list =  recursive_glob_files(task2_train_data_path, '.nii.gz')

task2_train_mask_data_path_list = [x for x in task2_train_data_path_list if x.endswith('mask.nii.gz')]

In [196]:
def calculate_filled_lesion_volume(nifti_file_path):
    img = nib.load(nifti_file_path)
    nifti_arr = img.get_fdata()
    fill_hole_nifit_arr = ndimage.binary_fill_holes(nifti_arr).astype(int)
    filled_lesion_volume = np.count_nonzero(fill_hole_nifit_arr)
    
    return filled_lesion_volume

In [197]:
def calculate_hole_lesion_volume(nifti_file_path):
    img = nib.load(nifti_file_path)
    nifti_arr = img.get_fdata()
    
    hole_lesion_volume = np.count_nonzero(nifti_arr)
    
    return hole_lesion_volume

In [198]:
total_dict = dict()

In [ ]:




from scipy import ndimage
a = np.zeros((5, 5), dtype=int)
a[1:4, 1:4] = 1
a[2,2] = 0
a

ndimage.binary_fill_holes(a).astype(int)

ndimage.binary_fill_holes(a, structure=np.ones((5,5))).astype(int)

task2_train_mask_data_path_list[0]

img = nib.load(task2_train_mask_data_path_list[0])
nifti_arr = img.get_fdata()

np.count_nonzero(nifti_arr)

np.count_nonzero(ndimage.binary_fill_holes(nifti_arr).astype(int))

nifti_arr.shape

nifti_arr[:,:,0].shape

single_axis_arr = nifti_arr[:,:,90]

single_axis_binary_fill_holes_arr = ndimage.binary_fill_holes(nifti_arr[:,:,90], structure=np.ones((197, 233)).astype(int))





np.count_nonzero(single_axis_arr)

np.count_nonzero(single_axis_binary_fill_holes_arr)

np.count_nonzero(single_axis_arr == single_axis_binary_fill_holes_arr)



In [256]:
import nibabel as nib
nii = nib.load(task2_train_mask_data_path_list[0])
sx, sy, sz = nii.header.get_zooms()
volume = sx * sy * sz

In [257]:
volume

1.0

In [199]:
for i, task2_train_mask_data_path in enumerate(task2_train_mask_data_path_list):
    temp_dict = dict()
    temp_dict['file_name'] = task2_train_mask_data_path
    temp_dict['filled_lesion_volume'] = calculate_filled_lesion_volume(task2_train_mask_data_path)
    temp_dict['hole_lesion_volume'] = calculate_hole_lesion_volume(task2_train_mask_data_path)
    
    total_dict[i] = temp_dict

In [242]:
df_task2_volume_size = pd.DataFrame(total_dict).T

In [252]:
df_task2_volume_size['subject'] = [x.split('\\')[-1].split('_')[0] for x in df_task2_volume_size['file_name']]


In [254]:
df_task2_volume_size

,file_name,filled_legion_volume,hole_legion_volume,subject
0,C:\Users\parkm\Desktop\github\analysis_nifti_f...,112488,112451,sub-r001s001
1,C:\Users\parkm\Desktop\github\analysis_nifti_f...,132613,132590,sub-r001s002
2,C:\Users\parkm\Desktop\github\analysis_nifti_f...,1482,1482,sub-r001s003
3,C:\Users\parkm\Desktop\github\analysis_nifti_f...,44701,44659,sub-r001s004
4,C:\Users\parkm\Desktop\github\analysis_nifti_f...,31843,31792,sub-r001s005
...,...,...,...,...
650,C:\Users\parkm\Desktop\github\analysis_nifti_f...,100200,100135,sub-r052s026
651,C:\Users\parkm\Desktop\github\analysis_nifti_f...,1760,1759,sub-r052s027
652,C:\Users\parkm\Desktop\github\analysis_nifti_f...,1038,1038,sub-r052s029
653,C:\Users\parkm\Desktop\github\analysis_nifti_f...,23624,23616,sub-r052s031


In [255]:
df_task2_volume_size.to_csv("df_task2_volume_size.csv", index=False, encoding='utf-8-sig')